In [3]:
import os
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
import re
from bs4 import BeautifulSoup
import requests

In [4]:
round_ = '17'
url = f'https://www.sportsbet.com.au/betting/australian-rules/afl/round-{round_}'
url

'https://www.sportsbet.com.au/betting/australian-rules/afl/round-17'

In [5]:
soup = BeautifulSoup(requests.get(url).text, 'html.parser')

In [6]:
tmp = soup.find_all('script')

In [7]:
game_urls = []
for i in tmp[:9]:
    url_find = re.compile(r'https://www.sportsbet.com.au/betting/australian-rules/afl/(.*?)\-\d{7}')
    game_url = url_find.search(i.text)
    game_urls.append(game_url.group())
game_urls

['https://www.sportsbet.com.au/betting/australian-rules/afl/richmond-v-sydney-7454960',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/western-bulldogs-v-collingwood-7454970',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/brisbane-v-west-coast-7454971',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/greater-western-sydney-v-hawthorn-7454980',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/st-kilda-v-melbourne-7454981',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/port-adelaide-v-gold-coast-7455001',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/geelong-v-north-melbourne-7455010',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/essendon-v-adelaide-7455011',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/fremantle-v-carlton-7455012']

In [8]:
r = re.compile(r'\/(\w*\-?\w*\-?\w*)\-v\-(\w*\-?\w*\-?\w*)\-')
game_list = []
for game in game_urls:
    game_list.append(r.findall(game))

In [9]:
game_list

[[('richmond', 'sydney')],
 [('western-bulldogs', 'collingwood')],
 [('brisbane', 'west-coast')],
 [('greater-western-sydney', 'hawthorn')],
 [('st-kilda', 'melbourne')],
 [('port-adelaide', 'gold-coast')],
 [('geelong', 'north-melbourne')],
 [('essendon', 'adelaide')],
 [('fremantle', 'carlton')]]

In [24]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.set_window_size(700,800)

In [25]:
disposals = ['To Get 15 or More Disposals',
            'To Get 20 or More Disposals',
            'To Get 25 or More Disposals',
            'To Get 30 or More Disposals',]
output = pd.DataFrame({},columns = {'teams','round','Player', 'disposals', 'odds'})

In [12]:
driver.get('https://www.sportsbet.com.au/betting/australian-rules/afl/richmond-v-sydney-7454960')

In [13]:
first_click = driver.find_element_by_xpath('(//*[@class="size14_f7opyze medium_f1wf24vo"])')
first_click.click()



In [14]:
for i in range(1,7):
            if driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze medium_f1wf24vo"])[{i}]').text == 'Disposal Markets':
                disposal_market = driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze medium_f1wf24vo"])[{i}]')
disposal_market.click()

In [23]:

for i in range(5,50):
    try:
        tmp = driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze bold_f1au7gae"])[{i}]').text
    except:
        continue
    if tmp in disposals:
        
        dis = driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze bold_f1au7gae"])[{i}]')

        dis.click()
        
        player = driver.find_element_by_xpath(f'(//*[@class="outcomeContainer_f1wc7xgg outcomeDetailsFirst_f162fqwy"])').text.split('\n')
        print(player)
        new_row = {'Player': player[0].lower(), 'teams': f'{game[0][0]} v {game[0][1]}', 'round':f'R{round_}', 'disposals':tmp,'odds':player[-1]}
        output.loc[len(output)] = new_row
        
        for j in range(1,50):
            try:
                
                tmp1 = driver.find_element_by_xpath(f'(//*[@class="outcomeContainer_f1wc7xgg"])[{j}]').text.split('\n')
                print(tmp1)
                new_row = {'Player': tmp1[0].lower(), 'teams': f'{game[0][0]} v {game[0][1]}', 'round':f'R{round_}', 'disposals':tmp,'odds':tmp1[-1]}
                output.loc[len(output)] = new_row
            except:
                continue
                
        dis.click()
        time.sleep(2)
            
output      

['Daniel Rioli', 'SUS']
['Dion Prestia', 'SUS']
['Nick Vlastuin', '1.09']
['Shai Bolton', '1.13']
['Dustin Martin', '1.14']
['Liam Baker', '1.18']
['Trent Cotchin', '1.24']
['Nathan Broad', '1.25']
['Noah Balta', '1.34']
['Jack Graham', '1.45']
['Toby Nankervis', '1.56']
['Sam Banks', '1.71']
['Jacob Bauer', '1.94']
['Kamdyn McIntosh', '1.94']
['Marlion Pickett', '2.08']
['Callum Mills', 'SUS']
['Jake Lloyd', 'SUS']
['Nick Blakey', '1.06']
['James Rowbottom', '1.11']
['Angus Sheldrick', '1.17']
['Robbie Fox', '1.30']
['Isaac Heeney', '1.37']
['Justin McInerney', '1.38']
['Oliver Florent', '1.40']
['Dane Rampe', '1.43']
['Braeden Campbell', '1.52']
['Harry Cunningham', '1.57']
['Tom Papley', '1.68']
['Tom Hickey', '1.95']
['Hayden McLean', '2.90']
['Tom McCartin', '3.15']
['Will Hayward', '3.35']
['Jacob Hopper', '1.29']
['Dion Prestia', '1.35']
['Daniel Rioli', '1.37']
['Shai Bolton', '1.57']
['Dustin Martin', '1.58']
['Nick Vlastuin', '1.65']
['Trent Cotchin', '1.96']
['Liam Baker', '

,disposals,teams,odds,Player,round
0,To Get 15 or More Disposals,aaa,SUS,daniel rioli,R17
1,To Get 15 or More Disposals,aaa,SUS,dion prestia,R17
2,To Get 20 or More Disposals,aaa,1.29,jacob hopper,R17
3,To Get 20 or More Disposals,aaa,1.35,dion prestia,R17
4,To Get 25 or More Disposals,aaa,1.24,tim taranto,R17
...,...,...,...,...,...
128,To Get 30 or More Disposals,aaa,9.25,oliver florent,R17
129,To Get 30 or More Disposals,aaa,13.00,james rowbottom,R17
130,To Get 30 or More Disposals,aaa,15.00,justin mcinerney,R17
131,To Get 30 or More Disposals,aaa,16.00,dane rampe,R17


In [ ]:
disposal_market.click()

In [26]:
for game, game_url in zip(game_list, game_urls):
    driver.get(game_url)
    try:
        first_click = driver.find_element_by_xpath('(//*[@class="size14_f7opyze medium_f1wf24vo"])')
        first_click.click()
        
        for i in range(1,7):
            if driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze medium_f1wf24vo"])[{i}]').text == 'Disposal Markets':
                disposal_market = driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze medium_f1wf24vo"])[{i}]')
        
        
        disposal_market.click()
        time.sleep(3)
        
        for i in range(5,50):
            try:
                tmp = driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze bold_f1au7gae"])[{i}]').text
            except:
                continue
            if tmp in disposals:

                dis = driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze bold_f1au7gae"])[{i}]')

                dis.click()

                player = driver.find_element_by_xpath(f'(//*[@class="outcomeContainer_f1wc7xgg outcomeDetailsFirst_f162fqwy"])').text.split('\n')
                print(player)
                new_row = {'Player': player[0].lower(), 'teams': f'{game[0][0]} v {game[0][1]}', 'round':f'R{round_}', 'disposals':tmp,'odds':player[-1]}
                output.loc[len(output)] = new_row

                for j in range(1,50):
                    try:

                        tmp1 = driver.find_element_by_xpath(f'(//*[@class="outcomeContainer_f1wc7xgg"])[{j}]').text.split('\n')
                        print(tmp1)
                        new_row = {'Player': tmp1[0].lower(), 'teams': f'{game[0][0]} v {game[0][1]}', 'round':f'R{round_}', 'disposals':tmp,'odds':tmp1[-1]}
                        output.loc[len(output)] = new_row
                    except:
                        continue
                
                dis.click()
                time.sleep(2)
            
    except:
        continue

['Dion Prestia', 'SUS']
['Daniel Rioli', 'SUS']
['Nick Vlastuin', '1.10']
['Shai Bolton', '1.13']
['Dustin Martin', '1.15']
['Liam Baker', '1.19']
['Trent Cotchin', '1.25']
['Nathan Broad', '1.26']
['Noah Balta', '1.36']
['Jack Graham', '1.42']
['Toby Nankervis', '1.55']
['Sam Banks', '1.68']
['Jacob Bauer', '1.93']
['Kamdyn McIntosh', '1.99']
['Marlion Pickett', '2.00']
['Callum Mills', 'SUS']
['Jake Lloyd', 'SUS']
['Nick Blakey', '1.06']
['James Rowbottom', '1.12']
['Angus Sheldrick', '1.17']
['Robbie Fox', '1.31']
['Isaac Heeney', '1.35']
['Justin McInerney', '1.39']
['Oliver Florent', '1.40']
['Dane Rampe', '1.42']
['Braeden Campbell', '1.52']
['Harry Cunningham', '1.53']
['Tom Papley', '1.68']
['Tom Hickey', '1.93']
['Hayden McLean', '2.95']
['Tom McCartin', '3.15']
['Will Hayward', '3.35']
['Jacob Hopper', '1.32']
['Dion Prestia', '1.35']
['Daniel Rioli', '1.37']
['Shai Bolton', '1.55']
['Dustin Martin', '1.61']
['Nick Vlastuin', '1.64']
['Liam Baker', '1.96']
['Trent Cotchin', '

['Jake Bowey', '1.21']
['Max Gawn', '1.46']
['Jake Lever', '1.56']
['Alex Neal-Bullen', '1.65']
['Judd McVee', '1.67']
['Brodie Grundy', '1.68']
['Brad Crouch', '1.08']
['Nasiah Wanganeen-Milera', '1.37']
['Jack Steele', '1.38']
['Mason Wood', '1.68']
['Rowan Marshall', '1.94']
['Marcus Windhager', '2.20']
['Callum Wilkie', '2.55']
['Jade Gresham', '2.75']
['Liam Stocker', '3.35']
['Jimmy Webster', '4.00']
['Mitchito Owens', '4.50']
['Dougal Howard', '6.25']
['Mattaes Phillipou', '9.50']
['Jack Viney', '1.12']
['Angus Brayshaw', '1.16']
['Lachie Hunter', '1.23']
['Christian Salem', '1.27']
['Ed Langdon', '1.52']
['Trent Rivers', '1.71']
['Steven May', '1.89']
['Tom Sparrow', '1.95']
['Jake Bowey', '2.20']
['Jake Lever', '3.10']
['Max Gawn', '3.45']
['Judd McVee', '4.25']
['Brodie Grundy', '4.30']
['Alex Neal-Bullen', '4.50']
['Jack Sinclair', '1.19']
['Brad Crouch', '1.46']
['Jack Steele', '2.18']
['Nasiah Wanganeen-Milera', '2.18']
['Mason Wood', '4.10']
['Marcus Windhager', '4.90']
[

['Archie Perkins', '4.25']
['Sam Durham', '4.30']
['Jayden Laverde', '4.50']
['Kyle Langford', '4.80']
['Jake Kelly', '5.20']
['Will Snelling', '10.00']
['Ben Keays', '1.08']
['Mitch Hinge', '1.72']
['Brodie Smith', '1.77']
['Rory Sloane', '1.85']
['Wayne Milera', '2.10']
['Chayce Jones', '2.20']
['Izak Rankine', '2.50']
['Josh Rachele', '2.95']
['Lachlan Sholl', '3.00']
['Ned McHenry', '3.05']
['Jake Soligo', '3.50']
['Josh Worrell', '3.75']
['Lachlan Murphy', '3.90']
["Reilly O'Brien", '4.30']
['Taylor Walker', '5.50']
['Riley Thilthorpe', '7.75']
['Darcy Parish', '1.17']
['Zach Merrett', '1.32']
['Dylan Shiel', '2.20']
['Ben Hobbs', '2.65']
['Nic Martin', '2.65']
['Jye Caldwell', '2.90']
['Mason Redman', '3.15']
['Jordan Ridley', '3.75']
['Andrew McGrath', '3.80']
['Dyson Heppell', '3.90']
['Jake Stringer', '8.25']
['Jake Kelly', '14.00']
['Jayden Laverde', '14.00']
['Kyle Langford', '16.00']
['Archie Perkins', '17.00']
['Sam Durham', '17.00']
['Jordan Dawson', '1.21']
['Rory Laird'

In [ ]:
for game, game_url in zip(game_list, game_urls):
    driver.get(game_url)
    try:
        first_click = driver.find_element_by_xpath('(//*[@class="size14_f7opyze medium_f1wf24vo"])')
        first_click.click()
        
        disposal_market = driver.find_element_by_xpath('(//*[@class="size14_f7opyze medium_f1wf24vo"])[5]')
        disposal_market.click()
        time.sleep(3)
        
        for i in range(5,9):
            dis = driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze bold_f1au7gae"])[{i}]')
            
            dis.click()
            
            tmp = driver.find_element_by_xpath(f'(//*[@class="outcomeContainer_f1wc7xgg outcomeDetailsFirst_f162fqwy"])').text.split('\n')
            new_row = {'Player': tmp[0].lower(), 'teams': f'{game[0][0]} v {game[0][1]}', 'round':f'R{round_}', 'disposals':disposals[i - 5],'odds':tmp[-1]}
            output.loc[len(output)] = new_row
            
            for j in range(50):
                try:
                    tmp1 = driver.find_element_by_xpath(f'(//*[@class="outcomeContainer_f1wc7xgg"])[{j}]').text.split('\n')
                    new_row = {'Player': tmp1[0].lower(), 'teams': f'{game[0][0]} v {game[0][1]}', 'round':f'R{round_}', 'disposals':disposals[i - 5],'odds':tmp1[-1]}
                    print(tmp1)
                    output.loc[len(output)] = new_row
                except:
                    continue

            dis.click()
            time.sleep(2)
            
    except:
        continue

In [27]:
output

,disposals,teams,odds,Player,round
0,To Get 15 or More Disposals,richmond v sydney,SUS,dion prestia,R17
1,To Get 15 or More Disposals,richmond v sydney,SUS,daniel rioli,R17
2,To Get 15 or More Disposals,richmond v sydney,1.10,nick vlastuin,R17
3,To Get 15 or More Disposals,richmond v sydney,1.13,shai bolton,R17
4,To Get 15 or More Disposals,richmond v sydney,1.15,dustin martin,R17
...,...,...,...,...,...
818,To Get 30 or More Disposals,fremantle v carlton,6.00,blake acres,R17
819,To Get 30 or More Disposals,fremantle v carlton,8.50,matthew kennedy,R17
820,To Get 30 or More Disposals,fremantle v carlton,10.50,adam saad,R17
821,To Get 30 or More Disposals,fremantle v carlton,11.00,nic newman,R17


In [28]:
output['home'] = output['teams'].apply(lambda x: x.split(' v ')[0])
output['away'] = output['teams'].apply(lambda x: x.split(' v ')[1])

In [29]:
team_mapping = {'richmond':'RCH','geelong':'GEE',
                'kangaroos':'NTH','port-adelaide':'POR',
                'gold-coast':'GCS','melbourne':'MEL',
                'greater-western-sydney':'GWS','hawthorn':'HAW',
                'carlton':'CAR','brisbane':'BRL',
                'collingwood':'COL','adelaide':'ADE',
                'fremantle':'FRE','western-bulldogs':'WBD',
                'sydney':'SYD','west-coast':'WCE',
                'st-kilda':'STK','essendon':'ESS'}

In [30]:
for key in team_mapping.keys():
     output.loc[output['home']==key, 'home'] = team_mapping[key]
for key in team_mapping.keys():
    output.loc[output['away']==key, 'away'] = team_mapping[key]     

In [31]:
output

,disposals,teams,odds,Player,round,home,away
0,To Get 15 or More Disposals,richmond v sydney,SUS,dion prestia,R17,RCH,SYD
1,To Get 15 or More Disposals,richmond v sydney,SUS,daniel rioli,R17,RCH,SYD
2,To Get 15 or More Disposals,richmond v sydney,1.10,nick vlastuin,R17,RCH,SYD
3,To Get 15 or More Disposals,richmond v sydney,1.13,shai bolton,R17,RCH,SYD
4,To Get 15 or More Disposals,richmond v sydney,1.15,dustin martin,R17,RCH,SYD
...,...,...,...,...,...,...,...
818,To Get 30 or More Disposals,fremantle v carlton,6.00,blake acres,R17,FRE,CAR
819,To Get 30 or More Disposals,fremantle v carlton,8.50,matthew kennedy,R17,FRE,CAR
820,To Get 30 or More Disposals,fremantle v carlton,10.50,adam saad,R17,FRE,CAR
821,To Get 30 or More Disposals,fremantle v carlton,11.00,nic newman,R17,FRE,CAR


In [33]:
output.to_csv('/Users/caseyh/Desktop/footyfootyfooty/sportsbet_odds/test_disposal_markets_r17.csv', index = False)